![GenAI Banner](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)



<p><font size="5" color='grey'> <b> Chat & Memory </b></font> </br></p>

---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()
# Bei Bedarf: Trennen zwischen Installationsname () und Importname (für Python) beide Angaben in Klammern
# install_packages([('markitdown[all]', 'markitdown'), 'langchain_chroma', ]

## 🔧 Globale Konstanten

Diese Konstanten werden im gesamten Notebook verwendet und können hier zentral angepasst werden.

In [ ]:
# ========================================
# GLOBALE KONSTANTEN
# ========================================

# LLM-Konfiguration
MODEL_NAME = "gpt-4o-mini"
TEMPERATURE = 0

# System-Prompt
SYSTEM_PROMPT = "Du bist ein hilfreicher und humorvoller KI-Assistent."

# Memory-Einstellungen
MAX_MESSAGES_BEFORE_TRIM = 10
MAX_TOKENS_FOR_TRIM = 1000
MESSAGES_TO_SUMMARIZE = 8
RECENT_MESSAGES_TO_KEEP = 2



# 1 | Intro
---


<p><font color='black' size="5">
Warum braucht KI ein Gedächtnis?
</font></p>

Large Language Models wie GPT sind von Natur aus **zustandslos** – sie verfügen über kein eingebautes Gedächtnis. Jede Anfrage wird isoliert verarbeitet, ohne Bezug zu vorherigen Interaktionen.

```
Ohne Memory:
User: "Mein Name ist Max"
AI: "Hallo Max!"
User: "Wie heiße ich?"
AI: "Das habe ich nicht gespeichert." ❌

Mit Memory:
User: "Mein Name ist Max"
AI: "Hallo Max!"
User: "Wie heiße ich?"
AI: "Du heißt Max!" ✅
```

<p><font color='black' size="5">
Memory-Typen im Überblick
</font></p>

Ein solches *Gedächtsnis* lässt sich unterschiedliche implementieren:

| Typ | Beschreibung | Beispiel | Speicherort | Technologie |
|-----|--------------|----------|-------------|-------------|
| **Kurzzeit-Memory** | Innerhalb einer Sitzung | ChatGPT erinnert sich an vorherige Nachrichten | RAM | Python Liste, LangGraph |
| **Memory-Management** | Optimierung langer Chats | Zusammenfassung, Trimming | RAM | Summarization, Sliding Window |
| **Externes Memory** | Wissensdatenbank | RAG-Systeme, Dokumentensuche | Datenbank | Vektordatenbank + Retrieval |


<p><font color='black' size="5">
Kontextfenster verstehen
</font></p>

**Wichtiges Konzept:** Das Kontextfenster bestimmt, wie viele Tokens (Wörter/Zeichen) ein LLM gleichzeitig verarbeiten kann.

```
GPT-4o mini:    128.000 Tokens (~96.000 Wörter)
Claude Sonnet:  200.000 Tokens (~150.000 Wörter)
```

**Problem:** Mehr Tokens ≠ Besseres Gedächtnis
- Kosten steigen linear
- Relevante Infos gehen in der Masse unter
- Längere Antwortzeiten

**Lösung:** Intelligentes Memory-Management statt einfach "alles reinpacken"




# 2 | Kurzzeit-Memory
---


Kurzzeit-Memory speichert den Gesprächsverlauf einer aktiven Sitzung und ermöglicht kontextbezogene Antworten.


## 2.1 | Chat mit manuellem Memory


<p><font color='black' size="5">
Basismodell
</font></p>

In [ ]:
# Importe
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# Prompt-Template mit Historie
prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{user_input}")
])

In [ ]:
# LLM
llm = ChatOpenAI(model=MODEL_NAME, temperature=TEMPERATURE)

In [ ]:
# Parser
parser = StrOutputParser()

In [ ]:
# Chain
chain = prompt | llm | parser

In [ ]:
# Chat-Funktion
def chat_with_memory(system_prompt, chat_history, user_input):
    """Führt eine Chat-Interaktion mit manueller Historien-Verwaltung durch."""

    # Chain aufrufen
    response = chain.invoke({
        'system_prompt': system_prompt,
        'chat_history': chat_history,
        'user_input': user_input
    })

    # Ausgabe
    mprint(f"### 🧑‍🦱 Mensch: \n{user_input}")
    mprint(f"### 🤖 KI: \n{response}\n")

    # ⭐ WICHTIG: Memory manuell aktualisieren
    chat_history.extend([HumanMessage(content=user_input), AIMessage(content=response)])

    return chat_history


<p><font color='black' size="5">
Memory in Aktion
</font></p>

In [ ]:
# Historie initialisieren
chat_history = []

# Konversation
chat_with_memory(SYSTEM_PROMPT, chat_history, "Mein Name ist Max")
chat_with_memory(SYSTEM_PROMPT, chat_history, "Ich mag Python-Programmierung")
chat_with_memory(SYSTEM_PROMPT, chat_history, "Weißt du noch, wie ich heiße und was ich mag?")

# Historie inspizieren
mprint("### 📝 Gespeicherte Nachrichten:")
mprint("---")
for msg in chat_history:
    mprint(f"  **{msg.type}**:   {msg.content}")

<p><font color='black' size="5">
Bewertung:
</font></p>

✅ Memory ist eine Liste von Nachrichten    
✅ Jede Nachricht wird zur Historie hinzugefügt   
✅ Bei jedem API-Call wird die komplette Historie mitgeschickt    
✅ Das LLM sieht den gesamten Kontext    

❌ Problem: Keine Session-Verwaltung für mehrere Benutzer    
❌ Problem: Manuelles Memory-Management fehleranfällig    
❌ Problem: Keine Persistenz (nach Programmneustart weg)    

**Lösung:** LangGraph mit eingebauter Persistence


## 2.2 | LangGraph



**LangGraph** ist ein Framework zur Erstellung zustandsbasierter, mehrstufiger KI-Anwendungen. Das Kernkonzept basiert auf einem **Graph**, der aus **Nodes** (Knoten) und **Edges** (Kanten) besteht.



+ **State** (Zustand): Zentraler Datenspeicher, der durch den gesamten Graph weitergegeben und aktualisiert wird. Enthält alle relevanten Informationen.
+ **Nodes** (Knoten): Funktionen, die Aufgaben ausführen (z.B. LLM-Aufruf, Datenverarbeitung). Jeder Node erhalt den State und gibt einen aktualisierten State zurück.
+ **Edges** (Kanten): Verbindungen zwischen Nodes, die den Workflow-Fluss definieren. Sie bestimmen, welcher Nodeals nächstes ausgeführt
wird.
+ **Conditional Edges**: Bedingte Verbindungen, die basierend auf dem State oder anderen Bedingungen entscheiden, welcher Node als nächstes kommt.
+ **Graph**: Die Gesamtstruktur aus Nodes und Edges, die einen zustandsbasierten Workflow für KI-Anwendungen definiert.

So entstehen flexible, intelligente Workflows für komplexe KI-Anwendungen.



<img src="https://raw.githubusercontent.com/ralf-42/Image/main/chat_memory_04.png" class="logo" width="850"/>


<p><font color='black' size="5">
Nodes - Kategorisierung nach Rolle
</font></p>

| Kategorie (funktional) | Beschreibung | Beispiel |
|------------------------|--------------|----------|
| **Start-Node** | Einstiegspunkt in den Graphen; keine eigene Logik, definiert den Beginn des Ablaufs. | `START` |
| **End-Node** | Markiert das Ende der Ausführung; erhält typischerweise keine weiteren Übergänge. | `END` |
| **Agent-Node** | Enthält LLM- oder Entscheidungslogik, die den Zustand auswertet und neue Nachrichten oder Aktionen generiert. | `assistant`, `planner` |
| **Tool-Node** | Ruft externe Funktionen, APIs oder Datenquellen auf und erweitert den Zustand um Ergebnisse. | `search`, `database_query` |
| **Evaluator-Node** | Bewertet oder filtert Zwischenergebnisse, steuert ggf. den weiteren Ablauf. | `check_quality`, `approve_or_retry` |
| **Router-Node** | Leitet basierend auf Bedingungen an verschiedene Pfade weiter (`add_conditional_edges`). | `route_by_intent` |
| **Custom-Node** | Benutzerdefinierte Verarbeitungslogik ohne vordefinierte Rolle. | `normalize_text`, `extract_metadata` |


<p><font color='black' size="5">
Namenskonventionstabelle - Vorschlag
</font></p>

| Ebene / Phase | Empfohlene Variable | Beschreibung | Beispiel |
|----------------|--------------------|---------------|-----------|
| **Graph-Definition** | `workflow` oder `graph_def` | Enthält die reine logische Definition des Graphen – also Nodes, Edges und Zustandsschema. | `workflow = StateGraph(state_schema=MessagesState)` |
| **Kompilierte Instanz** | `compiled_workflow` oder `runner` | Das lauffähige Objekt nach dem Kompilieren; enthält den initialisierten Zustand und Checkpointer. | `runner = workflow.compile(checkpointer=memory)` |
| **Zustand** | `state` | Repräsentiert den aktuellen Datenzustand während der Ausführung (z. B. Nachrichten, Kontext, Variablen). | `state = {"messages": [...]} ` |
| **Lauf / Ergebnis** | `result` oder `final_state` | Rückgabe nach der Ausführung oder nach Erreichen von `END`. | `result = runner.invoke(state)` |
| **Speicher / Persistenz** | `checkpointer` oder `memory` | Objekt zur Verwaltung von Zwischenzuständen und Verlauf. | `memory = MemorySaver()` |
| **Teilgraph / Subworkflow** | `subgraph` | Optionaler Untergraph für modularisierte Logik. | `subgraph = StateGraph(state_schema=TaskState)` |
| **Agentenknoten** | `agent_node` oder `agent_fn` | Node-Funktion mit LLM- oder Entscheidungslogik. | `graph.add_node("agent", agent_node)` |
| **Toolknoten** | `tool_node` oder `tool_fn` | Node-Funktion, die externe Tools oder APIs aufruft. | `graph.add_node("search", tool_fn)` |



<p><font color='black' size="5">
Funktionsstruktur
</font></p>

<img src="https://raw.githubusercontent.com/ralf-42/Image/main/chat_memory_03.png" class="logo" width="500"/>

<p><font color='black' size="5">
Basismodell
</font></p>

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
# LLM initialisieren
llm = ChatOpenAI(model=MODEL_NAME, temperature=TEMPERATURE)

<p><font color='black' size="5">
StateGraph definieren
</font></p>

In [ ]:
# Node: Ruft das Modell auf
def agent_node(state: MessagesState):
    """ Diese Funktion wird bei jedem Chat-Schritt aufgerufen. Der 'state' enthält automatisch alle bisherigen Nachrichten.  """
    # System-Prompt hinzufügen
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT}
    ] + state["messages"]

    # LLM aufrufen
    response = llm.invoke(messages)

    # Rückgabe wird automatisch zu state["messages"] hinzugefügt
    return {"messages": [response]}

In [ ]:
# Graph-Workflow erstellen
workflow = StateGraph(state_schema=MessagesState)

# Dieser Knoten führt die Funktion "agent_node" aus, wenn er aufgerufen wird
# "agent_node" ist typischerweise die Funktion, die das LLM aufruft
workflow.add_node("agent", agent_node)

# Startkante definieren
workflow.add_edge(START, "agent")

# Endkante definieren
workflow.add_edge("agent", END)

# MemorySaver speichert den Konversations-Zustand zwischen verschiedenen Aufrufen,
# Ermöglicht es dem Agenten, sich an frühere Nachrichten zu "erinnern"
memory = MemorySaver()

# Graph kompilieren
# Wandelt die Workflow-Definition in eine ausführbare Anwendung um
runner =  workflow.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display

# Visualisiere als PNG
display(Image(app.get_graph().draw_mermaid_png()))

**Wichtig:** Der `MessagesState` verwaltet die Nachrichten automatisch!


In [ ]:
def chat(thread_id, user_input):
    """Chattet mit dem Bot in einem bestimmten Thread."""

    # Config mit Thread-ID (wie Session-ID)
    config = {"configurable": {"thread_id": thread_id}}

    # Input vorbereiten
    input_message = {"messages": [HumanMessage(content=user_input)]}

    # Graph aufrufen - Memory wird automatisch verwaltet!
    result = runner.invoke(input_message, config=config)

    # Ausgabe
    mprint(f"**🧑‍🦱 [Thread: {thread_id}] Mensch:**  \n{user_input}")
    mprint(f"**🤖 [Thread: {thread_id}] KI:**  \n{result['messages'][-1].content}\n")

    return result['messages'][-1].content


<p><font color='black' size="5">
Einzelne Thread/Konversation
</font></p>

In [ ]:
# Konversation starten
chat("Max_session", "Hallo! Mein Name ist Max.")
chat("Max_session", "Wieviele Tage hat das Jahr?.")
chat("Max_session", "Weißt du noch, wie ich heiße und wo ich wohne?")
pass


<p><font color='black' size="5">
Multi-User (Parallele Threads)
</font></p>

In [ ]:
# Thread 1: Max
chat("thread_max", "Ich mag Python-Programmierung!")

# Thread 2: Emma (komplett separate Konversation)
chat("thread_emma", "Ich interessiere mich für Machine Learning!")

# Zurück zu Thread 1 - Memory bleibt erhalten!
chat("thread_max", "Was war nochmal mein Interesse?")

# Zurück zu Thread 2
chat("thread_emma", "An welchem Thema bin ich interessiert?")
print()

✅ **Jeder Thread hat seine eigene, isolierte Historie!**




<p><font color='black' size="5">
Thread-Historie ansehen
</font></p>

In [ ]:
def show_thread_history(thread_id):
    """Zeigt die komplette Historie eines Threads."""

    config = {"configurable": {"thread_id": thread_id}}
    state = runner.get_state(config)

    mprint(f"### 📝 Thread '{thread_id}' - {len(state.values['messages'])} Nachrichten:\n")
    mprint("---")

    for msg in state.values["messages"]:
        role = "🧑‍🦱" if msg.type == "human" else "🤖"
        mprint(f"{role} {msg.type.upper()}: {msg.content}")

# Beispiel
show_thread_history("thread_max")

In [ ]:
show_thread_history("thread_emma")

<p><font color='black' size="5">
💼 Für Production: Automatische Thread-IDs
</font></p>


In echten Anwendungen sollte man eindeutige Thread-IDs automatisch generieren:

In [ ]:
import uuid
from datetime import datetime

def create_new_conversation(user_name: str) -> str:
    """
    Erstellt eine neue Konversations-ID für einen User.

    Format: username_timestamp_uuid
    Beispiel: Max_20250127_a1b2c3d4
    """
    timestamp = datetime.now().strftime("%Y%m%d")
    unique_id = uuid.uuid4().hex[:8]
    thread_id = f"{user_name}_{timestamp}_{unique_id}"
    return thread_id

# Beispiel: Neue Konversationen für verschiedene User
thread_max = create_new_conversation("Max")
thread_emma = create_new_conversation("emma")

print(f"Max's neue Konversation: {thread_max}")
print(f"Emma's neue Konversation: {thread_emma}")

# Verwendung
chat(thread_max, "Hallo, ich bin Max!")
chat(thread_emma, "Hi, ich bin Emma!")
print()

In [ ]:
chat(thread_emma, "Wie ist mein Name?")
print()


# 3 | Memory Management Strategien
---

**Problem:** Lange Konversationen sprengen das Kontextfenster und werden teuer.

**Lösung:** Intelligentes Memory-Management


<p><font color='black' size="5">
Funktionsstruktur
</font></p>

<img src="https://raw.githubusercontent.com/ralf-42/Image/main/chat_memory_02.png" class="logo" width="850"/>


## 3.1 | Trimming

**Strategie:** Behalte nur die letzten *n* Token oder *n* Nachrichten.

<p><font color='black' size="5">
Tokenbasiertes Trimming
</font></p>

In [ ]:
from langchain_core.messages import trim_messages

# Trimming: Nur die letzten n Token behalten
def agent_node_with_trimming(state: MessagesState):
    """Ruft das Modell mit getrimmter Historie auf."""

    # System-Prompt
    system_msg = {"role": "system", "content": SYSTEM_PROMPT}

    # Nur die letzten Nachrichten basierend auf MAX_TOKENS_FOR_TRIM
    trimmed = trim_messages(
        state["messages"],
        max_tokens=MAX_TOKENS_FOR_TRIM,
        strategy="last",
        include_system=False
    )

    messages = [system_msg] + trimmed
    response = llm.invoke(messages)

    return {"messages": [response]}

# Graph mit Trimming
workflow_trimmed = StateGraph(state_schema=MessagesState)
workflow_trimmed.add_node("agent", agent_node_with_trimming)
workflow_trimmed.add_edge(START, "agent")
workflow_trimmed.add_edge("agent", END)

app_trimmed = workflow_trimmed.compile(checkpointer=MemorySaver())


<p><font color='black' size="5">
Nachrichtenbasiertes Trimming
</font></p>


```
from langchain_core.messages import trim_messages

# Anzahl der Nachrichten begrenzen
trimmed_messages = trim_messages(
    messages,
    max_tokens=10,           # ← Anzahl der Nachrichten
    strategy="last",
    token_counter=len,       # ← len() zählt Nachrichten statt Token
    include_system=True      # System-Nachricht immer behalten
)
```



## 3.2 | Summarization

**Strategie:** Fasse alte Nachrichten zusammen, behalte nur Zusammenfassung + letzte Nachrichten.

<p><font color='black' size="5">
Grundprinzip
</font></p>

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Zusammenfassungs-Prompt
summarize_prompt = ChatPromptTemplate.from_messages([
    ("system", "Fasse die folgende Konversation in 2-3 Sätzen zusammen. Behalte wichtige Fakten."),
    ("human", "{conversation}")
])

def summarize_conversation(messages):
    """Erstellt eine Zusammenfassung der Nachrichten."""

    # Konversation als Text formatieren
    conversation_text = "\n".join([
        f"{msg.type}: {msg.content}" for msg in messages
    ])

    # Zusammenfassung erstellen
    summary_chain = summarize_prompt | llm | StrOutputParser()
    summary = summary_chain.invoke({"conversation": conversation_text})

    return summary

# Beispiel
old_messages = [
    HumanMessage(content="Mein Name ist Max"),
    AIMessage(content="Hallo Max!"),
    HumanMessage(content="Ich wohne in Köln"),
    AIMessage(content="Köln ist toll!"),
]

summary = summarize_conversation(old_messages)
mprint(f"###📝 Zusammenfassung: \n{summary}")

<p><font color='black' size="5">
Basismodell mit LangGraph
</font></p>

In [ ]:
from langchain_core.messages import SystemMessage

class StateWithSummary(MessagesState):
    """State mit zusätzlichem Summary-Feld."""
    summary: str = ""

def agent_node_with_summary(state: StateWithSummary):
    """Nutzt Zusammenfassung statt alter Nachrichten."""

    # Wenn mehr als MAX_MESSAGES_BEFORE_TRIM Nachrichten: Summarize
    if len(state["messages"]) > MAX_MESSAGES_BEFORE_TRIM:
        # Erste MESSAGES_TO_SUMMARIZE Nachrichten zusammenfassen
        to_summarize = state["messages"][:MESSAGES_TO_SUMMARIZE]
        summary = summarize_conversation(to_summarize)

        # Nur Summary + letzte RECENT_MESSAGES_TO_KEEP Nachrichten verwenden
        messages = [
            SystemMessage(content=f"{SYSTEM_PROMPT}\n\nBisheriger Kontext: {summary}"),
        ] + state["messages"][-RECENT_MESSAGES_TO_KEEP:]

        if DEBUG_MODE:
            print(f"📝 Zusammenfassung erstellt: {summary}")
    else:
        messages = [SystemMessage(content=SYSTEM_PROMPT)] + state["messages"]

    response = llm.invoke(messages)
    return {"messages": [response]}

# Graph mit Summary-Funktion erstellen
workflow_summary = StateGraph(state_schema=StateWithSummary)
workflow_summary.add_node("agent", agent_node_with_summary)
workflow_summary.add_edge(START, "agent")
workflow_summary.add_edge("agent", END)

# Mit Memory kompilieren
app_summary = workflow_summary.compile(checkpointer=MemorySaver())

<p><font color='black' size="5">
Einsatz
</font></p>

In [ ]:
def chat_with_summary(thread_id, user_input):
    """Chattet mit dem Bot unter Verwendung der Summary-Strategie."""

    config = {"configurable": {"thread_id": thread_id}}
    input_message = {"messages": [HumanMessage(content=user_input)]}

    result = app_summary.invoke(input_message, config=config)

    mprint(f"### 🧑‍🦱 Mensch: \n{user_input}")
    mprint(f"### 🤖 KI: \n{result['messages'][-1].content}\n")

    return result['messages'][-1].content


In [ ]:
# Beispiel: Lange Konversation (mehr als MAX_MESSAGES_BEFORE_TRIM)
thread = "summary_test"

# Erste Nachrichten
chat_with_summary(thread, "Mein Name ist Max")
chat_with_summary(thread, "Ich wohne in Köln")
chat_with_summary(thread, "Ich arbeite als Entwickler")
chat_with_summary(thread, "Mein Hobby ist Wandern")
chat_with_summary(thread, "Ich mag Python")

# Viele weitere Nachrichten hinzufügen...
for i in range(6):
    chat_with_summary(thread, f"Dies ist Test-Nachricht Nummer {i+1}")

# Nach vielen Nachrichten - sollte trotzdem wichtige Infos kennen
chat_with_summary(thread, "Wie heiße ich und wo wohne ich?")
print()


## 3.3 | Wann welche Strategie?

| Szenario | Empfehlung |
|----------|-----------|
| **Kurze Chats (<10 Nachrichten)** | Keine Optimierung nötig |
| **Support-Bot (20-50 Nachrichten)** | Trimming (letzte 20) |
| **Langzeit-Assistent (100+ Nachrichten)** | Summarization + Trimming |
| **Fakten-basierte Chats** | Externe Memory (RAG) |

---


# 4 | Externes Memory (RAG Ausblick)
---



**Problem:** Auch mit Summarization sind sehr lange Konversationen oder große Wissensbasen problematisch.

**Lösung:** Retrieval-Augmented Generation (RAG)

<p><font color='black' size="5">
Konzept
</font></p>

Statt alles im Chat-Verlauf zu speichern:
1. **Speichere Fakten in Vektordatenbank** (Chroma, Pinecone, FAISS)
2. **Bei Bedarf relevante Infos abrufen**
3. **Nur relevantes zur LLM-Anfrage hinzufügen**

```
User: "Was war mein erstes Projekt?"

1. Semantische Suche in Vektordatenbank
2. Finde: "Projekt X von 2020..."
3. Füge nur relevante Infos zum Prompt hinzu
4. LLM antwortet mit Kontext
```



<p><font color='black' size="5">
Unterschied zu Chat-Memory
</font></p>

| Aspekt | Chat-Memory | Externes Memory (RAG) |
|--------|-------------|----------------------|
| **Zweck** | Gesprächskontext | Wissensdatenbank |
| **Speicherart** | Chronologisch | Semantisch durchsuchbar |
| **Größe** | Klein (KB-MB) | Groß (GB-TB) |
| **Zugriff** | Sequenziell | Relevanz-basiert |
| **Multi-User** | Getrennt pro User | Geteilt |
| **Beispiel** | "Du heißt Max" | "Python ist eine Programmiersprache" |


<p><font color='black' size="5">
Vereinfachtes Beispiel
</font></p>


In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Vektordatenbank erstellen
vectorstore = Chroma.from_texts(
    texts=[
        "Max hat 2020 ein Chatbot-Projekt gemacht",
        "Maxs Lieblingssprache ist Python",
        "Max wohnt in Köln"
    ],
    embedding=OpenAIEmbeddings()
)

# Bei Bedarf Erinnerungen abrufen
def retrieve_memory(query: str, k: int = 2):
    """Sucht relevante Erinnerungen in der Vektordatenbank."""
    docs = vectorstore.similarity_search(query, k=k)
    return "\n".join([doc.page_content for doc in docs])

# In LangGraph integrieren
def agent_node_with_rag(state: MessagesState):
    """Nutzt externe Wissensdatenbank (RAG) zusätzlich zur Chat-Historie."""

    user_query = state["messages"][-1].content

    # Relevante Erinnerungen abrufen
    memories = retrieve_memory(user_query)

    if DEBUG_MODE:
        print(f"🔍 Abgerufene Erinnerungen: {memories}")

    # Als Kontext hinzufügen
    messages = [
        SystemMessage(content=f"{SYSTEM_PROMPT}\n\nRelevante Informationen: {memories}")
    ] + state["messages"]

    response = llm.invoke(messages)
    return {"messages": [response]}

# Graph mit RAG erstellen
workflow_rag = StateGraph(state_schema=MessagesState)
workflow_rag.add_node("agent", agent_node_with_rag)
workflow_rag.add_edge(START, "agent")
workflow_rag.add_edge("agent", END)

runner_rag = workflow_rag.compile(checkpointer=MemorySaver())

<p><font color='black' size="5">
Anwendung
</font></p>

In [ ]:
def chat_with_rag(thread_id: str, user_input: str):
    """Chattet mit dem Bot unter Verwendung von RAG."""

    config = {"configurable": {"thread_id": thread_id}}
    input_message = {"messages": [HumanMessage(content=user_input)]}

    result = runner_rag.invoke(input_message, config=config)

    print(f"🧑‍🦱 Mensch: {user_input}")
    print(f"🤖 KI: {result['messages'][-1].content}\n")

    return result['messages'][-1].content

# Beispiel: Die KI kann auf externe Wissensbasis zugreifen
chat_with_rag("rag_test", "Wann hat Max sein Chatbot-Projekt gemacht?")
chat_with_rag("rag_test", "Welche Programmiersprache mag Max?")
chat_with_rag("rag_test", "Fasse zusammen, was du über Max weißt.")

# 5 | Mehr Tokens ≠ Bessere Chat-Memory
---




**Mythos:** "Größeres Kontextfenster = Besseres Memory"

**Realität:** Nicht unbedingt!

<p><font color='black' size="5">
Warum mehr Tokens problematisch sind
</font></p>


**Rauschen und Irrelevanz**
```
10 Nachrichten: ✅ Alles relevant
100 Nachrichten: ⚠️ 80% irrelevant für aktuelle Frage
1000 Nachrichten: ❌ Wichtige Infos gehen unter
```

**Abnehmende Aufmerksamkeit**

LLMs nutzen Attention-Mechanismen:
- Bei kurzen Kontexten: Hohe Aufmerksamkeit auf jedes Detail
- Bei langen Kontexten: Attention verteilt sich dünn
- Wichtige Infos in der Mitte werden oft "übersehen" ("Lost in the middle" Problem)

**Kosten**

```
GPT-4o mini Input-Kosten (1M Tokens)

Chat mit 1.000 Tokens:  $0.15 / 1000 = $0.00015
Chat mit 10.000 Tokens: $0.15 / 100 = $0.0015    (10x teurer!)
Chat mit 100.000 Tokens: $0.15 / 10 = $0.015     (100x teurer!)
```

**Latenz**

Mehr Tokens = Längere Verarbeitung = Langsamere Antworten



<p><font color='black' size="5">
Best Practices
</font></p>


✅ **DO:**
- Intelligentes Trimming verwenden
- Alte Nachrichten zusammenfassen
- RAG für Faktenwissen nutzen
- Nur relevante Infos zum Context hinzufügen

❌ **DON'T:**
- Einfach alles in den Context packen
- "Mehr Tokens lösen das Problem" denken
- Memory-Management ignorieren

**Fazit**

> Ein großes Kontextfenster allein löst nicht die Herausforderungen von Chat & Memory. Entscheidend sind clevere Strategien, um Informationen zu selektieren, zusammenzufassen und zielgerichtet einzusetzen.



# A | Aufgaben
---


Die Aufgabenstellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.



<p><font color='black' size="5">
Aufgabe 1: Multi-User Chatbot mit LangGraph
</font></p>

**Schwierigkeit:** ⭐⭐

Erstellen Sie einen Chatbot mit LangGraph, der:
- Mindestens 3 verschiedene User-Threads verwaltet
- Jeden User beim Namen kennt
- Die Interessen jedes Users speichert
- Beim Thread-Wechsel den Kontext behält

**Bonus:** Implementieren Sie eine Funktion, die alle aktiven Threads auflistet.


<p><font color='black' size="5">
Aufgabe 2: Memory mit Trimming-Strategie
</font></p>

**Schwierigkeit:** ⭐⭐⭐

Implementieren Sie einen Chatbot, der:
1. Automatisch auf die letzten 10 Nachrichten trimmt
2. Dem User anzeigt, wenn Nachrichten "vergessen" wurden
3. Eine Zusammenfassung der vergessenen Nachrichten erstellt

**Erwartetes Verhalten:**
```
[Nach 12 Nachrichten]
🤖: "Hinweis: Ich habe unsere ersten Nachrichten zusammengefasst, um den Überblick zu behalten. Zusammenfassung: [...]"
```
